In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [2]:
!git clone https://github.com/Ayush-Patel-10/coco-caption

Cloning into 'coco-caption'...
remote: Enumerating objects: 801, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 801 (delta 4), reused 0 (delta 0), pack-reused 790
Receiving objects: 100% (801/801), 128.17 MiB | 33.18 MiB/s, done.
Resolving deltas: 100% (459/459), done.
Updating files: 100% (48/48), done.


In [3]:
%cd /content/coco-caption

/content/coco-caption


In [4]:
%matplotlib inline
from pycocotools.coco import COCO
#from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [5]:
# set up file names and pathes
dataDir='.'
dataType='val2014'
algName = 'fakecap'
annFile='%s/annotations/captions_%s.json'%(dataDir,dataType)
subtypes=['results', 'evalImgs', 'eval']
[resFile, evalImgsFile, evalFile]= \
['%s/results/captions_%s_%s_%s.json'%(dataDir,dataType,algName,subtype) for subtype in subtypes]

# download Stanford models
!./get_stanford_models.sh

Downloading...
--2023-12-05 02:09:11--  http://nlp.stanford.edu/software/stanford-corenlp-full-2015-12-09.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2015-12-09.zip [following]
--2023-12-05 02:09:12--  https://nlp.stanford.edu/software/stanford-corenlp-full-2015-12-09.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2015-12-09.zip [following]
--2023-12-05 02:09:12--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2015-12-09.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.

In [6]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
0:00:00.381473
creating index...
index created!
Loading and preparing results...     
DONE (t=0.01s)
creating index...
index created!


In [7]:
#__author__ = 'tylin'
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
# from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

class COCOEvalCap:
    def __init__(self, coco, cocoRes):
        print('ullu')
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.coco = coco
        self.cocoRes = cocoRes
        self.params = {'image_id': coco.getImgIds()}

    def evaluate2(self, gts, res):

        print('tokenizationme...')

        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            # (Cider(), "CIDEr"),
            (Spice(), "SPICE")
        ]

        # =================================================
        # Compute scores
        # =================================================
        for scorer, method in scorers:
            print('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, gts.keys(), m)
                    print("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, gts.keys(), method)
                print("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def evaluate(self):
        print("sdfsadf")
        print('d')
        imgIds = self.params['image_id']
        # imgIds = self.coco.getImgIds()
        gts = {}
        res = {}
        a=0
        for imgId in imgIds:
            gts[imgId] = self.coco.imgToAnns[imgId]
            res[imgId] = self.cocoRes.imgToAnns[imgId]
            if(a<1):
              print('')
              print(self.coco.imgToAnns[imgId])
              print(self.cocoRes.imgToAnns[imgId])
              a+=1
              print('')
        # =================================================
        # Set up scorers
        # =================================================
        print('tokenizationme...')

        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            # (Cider(), "CIDEr"),
            (Spice(), "SPICE")
        ]

        # =================================================
        # Compute scores
        # =================================================
        for scorer, method in scorers:
            print('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, gts.keys(), m)
                    print("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, gts.keys(), method)
                print("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

print("Done")

Done


# resutls on our dataset

In [8]:
with open('/content/drive/My Drive/Image-captioning-Researc-paper-implimentation-master/predictedCaption.json', 'r') as file:
  jfile = json.load(file)
with open('/content/drive/My Drive/Image-captioning-Researc-paper-implimentation-master/annotation.json', 'r') as file:
  cfile = json.load(file)

print("Files loaded")

Files loaded


In [9]:
gts={}
res = {}

for i in range(len(jfile)):

  tempResList = []

  image_id=jfile[i]['image_id']
  caption = jfile[i]['caption']
  dict1 = {'image_id' : image_id, 'id' : i, 'caption' : caption}
  tempResList.append(dict1)
  res.update({image_id: tempResList})

  tempGts = []
  for dicta in cfile:
    if(int(dicta['id']) == image_id):
      dict2 = {'image_id': image_id, 'id':i, 'caption': dicta['caption']}
      tempGts.append(dict2)
  gts.update({image_id: tempGts})
  if(i%100==0):
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [10]:
len(res), len(gts)

(7848, 7848)

In [11]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate2(gts, res)

ullu
tokenizationme...
setting up scorers...
computing Bleu score...
{'testlen': 90010, 'reflen': 82437, 'guess': [90010, 82162, 74314, 66482], 'correct': [22585, 4893, 1220, 323]}
ratio: 1.0918640901536798
Bleu_1: 0.251
Bleu_2: 0.122
Bleu_3: 0.063
Bleu_4: 0.033
computing METEOR score...
METEOR: 0.104
computing Rouge score...
ROUGE_L: 0.239
computing SPICE score...
SPICE: 0.133


In [12]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print('%s: %.2f'%(metric, score*100))

Bleu_1: 25.09
Bleu_2: 12.22
Bleu_3: 6.26
Bleu_4: 3.30
METEOR: 10.39
ROUGE_L: 23.86
SPICE: 13.27


In [15]:
# save evaluation results to ./results folder
json.dump(cocoEval.evalImgs, open(evalImgsFile, 'w'))
json.dump(cocoEval.eval,     open(evalFile, 'w'))